In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import json
import random
import pickle

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
intents = json.load(open("intents.json"))

In [3]:
words = []
classes = []
docs = []
ignore_tokens = ['?', '!', '.', ',']

In [4]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        docs.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [5]:
lematizer = WordNetLemmatizer()

In [6]:
words = [lematizer.lemmatize(word.lower()) for word in words if word not in ignore_tokens]

In [7]:
words = sorted(set(words))

In [8]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [9]:
training = []
output_empty = [0] * len(classes)

In [10]:
for doc in docs:
    bag = []
    word_patters = doc[0]
    word_patters = [lematizer.lemmatize(word.lower()) for word in word_patters]
    for word in words:
        bag.append(1) if word in word_patters else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [11]:
random.shuffle(training)
training = np.array(training)

C:\Users\vedan\AppData\Local\Temp/ipykernel_11664/2739997722.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [12]:
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [13]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [14]:
model = Sequential([
    layers.Dense(128, input_shape=(len(train_x[0]), ), activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(train_y[0]), activation="softmax")
])
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["acc"]
)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               22656     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
Total params: 31,432
Trainable params: 31,432
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_x, train_y, epochs=100)

Epoch 1/100
4/4 [==============================] - 1s 3ms/step - loss: 1.9833 - acc: 0.3445
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 1.8508 - acc: 0.4902
Epoch 3/100
4/4 [==============================] - 0s 3ms/step - loss: 1.6834 - acc: 0.5679
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 1.5889 - acc: 0.5371
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 1.4723 - acc: 0.5574
Epoch 6/100
4/4 [==============================] - 0s 3ms/step - loss: 1.3575 - acc: 0.5397
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 1.2925 - acc: 0.5335
Epoch 8/100
4/4 [==============================] - 0s 3ms/step - loss: 1.2349 - acc: 0.5449
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.1853 - acc: 0.5397
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 1.1221 - acc: 0.5485
Epoch 11/100
4/4 [==============================] - 0s 3ms/step - loss: 1.0492 

In [17]:
model.save("./models/bot_V1.model")

INFO:tensorflow:Assets written to: ./models/bot_V1.model\assets


In [18]:
model = keras.models.load_model("models/bot_V1.model")
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

In [19]:
def preprocess(item):
    tokens = nltk.word_tokenize(item)
    tokens = [lematizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(sentence):
    sentence_words = preprocess(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    
    return np.array(bag)


In [20]:
def predict_class(sentence, tolerance=0.7):
    bow = bag_of_words(sentence)
    prediction = model(np.array([bow]))[0]
    max_value = max(prediction).numpy()
    res = np.where(prediction.numpy() == max_value)[0][0]
    if max_value >= tolerance:
        return res
    else:
        return -1

In [21]:
predict_class("Who made you")

-1